In [ ]:
import numpy as np
import rasterio
import matplotlib.pyplot as plt

# --- helpers (simples et autonomes) ---

def to_nan(arr, nodata_values=(-9999, -32768, -3.4028235e38)):
    out = arr.astype('float32', copy=True)
    for nd in nodata_values:
        np.putmask(out, out == nd, np.nan)
    return out

def pad_right(img, ncols=1, fill_value=np.nan):
    pad = ((0, 0), (0, ncols)) if img.ndim == 2 else ((0, 0), (0, 0), (0, ncols))
    return np.pad(img, pad, mode="constant", constant_values=fill_value)

def pad_bottom(img, nrows=1, fill_value=np.nan):
    pad = ((0, nrows), (0, 0)) if img.ndim == 2 else ((0, 0), (0, nrows), (0, 0))
    return np.pad(img, pad, mode="constant", constant_values=fill_value)

def align_by_padding_2d(a, b, fill_value=np.nan):
    """Pad a et/ou b (2D) à droite/en bas pour obtenir la même taille."""
    Ha, Wa = a.shape
    Hb, Wb = b.shape
    if Ha < Hb:
        a = pad_bottom(a, Hb - Ha, fill_value)
    elif Hb < Ha:
        b = pad_bottom(b, Ha - Hb, fill_value)
    if Wa < Wb:
        a = pad_right(a, Wb - Wa, fill_value)
    elif Wb < Wa:
        b = pad_right(b, Wa - Wb, fill_value)
    return a, b

def clip_percentiles(x, pmin=2, pmax=98):
    vmin = np.nanpercentile(x, pmin)
    vmax = np.nanpercentile(x, pmax)
    return np.clip(x, vmin, vmax), (vmin, vmax)

# --- fonction principale ---

def show_band_difference(path_img1: str,
                         path_img2: str,
                         band: int = 1,
                         nodata_values = (-9999, -32768, -3.4028235e38),
                         percentile_clip: tuple[int,int] | None = (2, 98),
                         cmap: str = "seismic"):
    """
    Lit la bande `band` (1-based) de deux images et affiche la différence (img2 - img1).
    - NoData -> NaN
    - Aligne les tailles par padding (droite/bas) si nécessaire
    - Affiche avec palette divergente, échelle symétrique
    """
    # lecture bandes (2D)
    with rasterio.open(path_img1) as s1:
        a = s1.read(band)  # 2D (H,W)
    with rasterio.open(path_img2) as s2:
        b = s2.read(band)

    # NoData -> NaN
    a = to_nan(a, nodata_values)
    b = to_nan(b, nodata_values)

    # alignement des tailles
    a, b = align_by_padding_2d(a, b, fill_value=np.nan)

    # différence
    diff = b - a  # img2 - img1

    # optionnel : clip pour l'affichage (évite les outliers)
    if percentile_clip is not None:
        pmin, pmax = percentile_clip
        # échelle symétrique : on clip |diff|
        vmax = np.nanpercentile(np.abs(diff), pmax)
        diff_disp = np.clip(diff, -vmax, vmax)
    else:
        diff_disp = diff
        vmax = np.nanmax(np.abs(diff))

    # affichage
    plt.figure(figsize=(8, 6))
    im = plt.imshow(diff_disp, cmap=cmap, vmin=-vmax, vmax=vmax)
    plt.title(f"Différence bande {band}  (img2 - img1)")
    plt.axis("off")
    cb = plt.colorbar(im, shrink=0.85)
    cb.set_label("Δ intensité (unités de la bande)")
    plt.tight_layout()
    plt.show()

    return diff  # retourne aussi la matrice de différence (avec NaN sur padding/NoData)

diff = show_band_difference(r"C:\Users\guigu\Documents\PRO\Stage_SERTIT\outil_detection_changement\data\raw\testnotebooks\t3\Subset_S1A_IW_GRDH_1SDV_20190702T092435_20190702T092500_027936_032774_949A_Orb_NR_Cal_Spk_TC.tif", r"C:\Users\guigu\Documents\PRO\Stage_SERTIT\outil_detection_changement\data\raw\testnotebooks\t3\Subset_S1A_IW_GRDH_1SDV_20190726T092436_20190726T092501_028286_033210_6DDD_Orb_NR_Cal_Spk_TC.tif", band=1)
